In [1]:
#pip install scikit-learn pandas numpy

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [4]:
#load disease_data csv file

from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/MyDrive/Omdena/NLP in drug prediction/synthetic_data.csv')
merge_demo_amos_v3 = pd.read_csv('/content/gdrive/MyDrive/Omdena/NLP in drug prediction/merge_demo_amos_v3.csv')

Mounted at /content/gdrive


In [7]:
# Convert symptoms to a structured format using tfidf
vectorizer_tfidf = TfidfVectorizer()
X = vectorizer_tfidf.fit_transform(df["Symptoms"])  # Convert symptoms into a numerical format

# Encode the target variable (disease)
y = df["Disease"].str.lower()

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(accuracy)


0.9878333333333333


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
###test merge_demo_amos_v3
disease_list_from_sythetic_data = df["Disease"].str.lower().unique().tolist()
merge_demo_amos_v3_filtered = merge_demo_amos_v3[merge_demo_amos_v3["disease_name_standardized"].isin(disease_list_from_sythetic_data)]
merge_demo_amos_v3_filtered.columns

Index(['Unnamed: 0', 'disease_name', 'description', 'symptoms', 'causes',
       'source', 'table_name', 'treatment', 'url', 'diagnosis',
       'medication_name', 'medication_url', 'dosage_url', 'prognosis',
       'research', 'complications', 'faq', 'transmission',
       'disease_name_standardized', 'source_standardized', 'symptoms_SciSpaCy',
       'diagnosis_SciSpaCy', 'medication_SciSpaCy', 'treatment_SciSpaCy',
       'treatment_BioBERT'],
      dtype='object')

In [9]:
new_data = merge_demo_amos_v3_filtered
# Handle missing values in symptoms column (replace NaN with an empty string)
new_data["symptoms_SciSpaCy"] = new_data["symptoms_SciSpaCy"].fillna("").astype(str)

# Convert symptoms into numerical features using the same vectorizer
X_new = vectorizer_tfidf.transform(new_data["symptoms_SciSpaCy"])


# Make predictions
new_data["Predicted Disease"] = clf.predict(X_new)

# Calculate accuracy
accuracy = accuracy_score(new_data["disease_name_standardized"], new_data["Predicted Disease"])

# Generate classification report
report = classification_report(new_data["disease_name_standardized"], new_data["Predicted Disease"])

#new_data[["disease_name_standardized","Predicted Disease"]]
new_data.to_csv("/content/gdrive/MyDrive/Omdena/NLP in drug prediction/test.csv", index = False)

<ipython-input-9-b87b3a09a716>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["symptoms_SciSpaCy"] = new_data["symptoms_SciSpaCy"].fillna("").astype(str)
<ipython-input-9-b87b3a09a716>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data["Predicted Disease"] = clf.predict(X_new)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramet

In [11]:
# Count correct predictions (numerator)
correct_predictions = (new_data["disease_name_standardized"] == new_data["Predicted Disease"]).sum()

# Total number of samples (denominator)
total_samples = len(new_data)

correct_predictions, total_samples, 33/278

(31, 278, 0.11870503597122302)